Reading from HYDAT for a test case

In [1]:
import sqlite3
import datetime

import pandas as pd

# Change to your own HYDAT 
obs_path = '/Users/kasrakeshavarz/Downloads/Hydat.sqlite3'

# read observed data
conn = sqlite3.connect(obs_path) # locate your HYDAT sqlite database

In [2]:
def get_the_daily_dataframe(
    df: pd.DataFrame,
    regex_str: str,
    col: str,
    *args,
    **kwargs,
) -> pd.DataFrame:
    
    # filter and trim columns
    df = df.filter(regex=regex_str, axis=1) # extract the columns
    df.columns = df.columns.str.replace(r'\D', '', regex=True) # remove non-digits
    df = df.stack(future_stack=True) # stack without dropping
    df.index.names = ['STATION_NUMBER', 'YEAR', 'MONTH', 'DAY'] # assign index names
    df = df.reset_index() # reset index to add another level
    df['DATE'] = pd.to_datetime(df[['YEAR', 'MONTH', 'DAY']].astype(str).agg('-'.join, axis=1), errors='coerce') # define date column
    df.drop(columns=['YEAR', 'MONTH', 'DAY'], inplace=True) # drop unnecessary columns
    df.dropna(subset=['DATE'], inplace=True) # remove invalid dates
    df.set_index(keys=['STATION_NUMBER', 'DATE'], drop=True, inplace=True) # set index levels
    df.columns = [col] # assing column name
    
    # pivot data to look nice
    df = df.unstack(level='STATION_NUMBER')
    df = df.reorder_levels(order=[1,0], axis=1)
    
    return df




def extract_station_daily_flow(
    station: str,
    connection: sqlite3.Connection,
    start_date: str = '1850-01-01',
    end_date: str = str(datetime.datetime.now().date()),
    *args,
    **kwargs,
) -> pd.DataFrame:
    
    '''
    This function simply extracts data from the HYDAT sqlite3 database
    '''
    
    # read station data
    df = pd.read_sql_query(f"SELECT * FROM DLY_FLOWS WHERE STATION_NUMBER LIKE '%{station}%'", connection)
    
    # set index
    df.set_index(keys=['STATION_NUMBER', 'YEAR', 'MONTH'], drop=True, inplace=True)
    
    # get the FLOW and FLAG
    df_flow = get_the_daily_dataframe(df, r'^FLOW\d', 'FLOW')
    df_flag = get_the_daily_dataframe(df, r'^FLOW_.', 'FLAG')
    
    df = pd.concat([df_flow, df_flag], 
                   axis=1)
    df.sort_index(axis=0, inplace=True)
    df.sort_index(axis=1, level=0, ascending=False, inplace=True)
    
    df = df.loc[start_date:end_date, :]
    
    return df


def extract_station_coords(
    station: str,
    connection: sqlite3.Connection,
) -> dict:
    '''
    returns `latitude` and `longitude` values for `station`
    '''
    
    # read the specs
    df = pd.read_sql_query(f"SELECT * FROM STATIONS WHERE STATION_NUMBER LIKE '%{station}%'", conn)
    
    # rename the columns to their lowercase equivalent
    df.rename(
        columns={
            'LATITUDE': 'latitude',
            'LONGITUDE': 'longitude',
        },
        inplace=True,
    )
    
    # making a dictionary out of the values
    vals_dict = df.loc[:, ['latitude', 'longitude']].to_dict(orient='list')
    
    # return the values in form of a dictionary
    vals_dict = {k:v[0] for k,v in vals_dict.items()}
    
    return vals_dict

In [3]:
df = extract_station_daily_flow('05BB001', connection=conn, end_date="2026-12-31")

In [4]:
coords = extract_station_coords(station='05BB001', connection=conn)

Check for missing time-steps

In [5]:
freq = '1D'  # set your expected step
df2 = df.asfreq(freq)  # inserts missing timestamps with NaNs
missing_timestamps = df2.index[df2.isna().any(axis=1)]
has_missing = len(missing_timestamps) > 0

In [6]:
has_missing

True

Since it is a real case, we will proceed with it as is. First, assuring missing time-stamps are included:

In [7]:
freq = '1D'
full_index = pd.date_range(df.index.min(), df.index.max(), freq=freq, tz=df.index.tz)
df_full = df.reindex(full_index)  # missing rows → NaN

In [8]:
df_full

STATION_NUMBER 05BB001      
                  FLOW  FLAG
1909-05-01         NaN  None
1909-05-02         NaN  None
1909-05-03         NaN  None
1909-05-04         NaN  None
1909-05-05         NaN  None
...                ...   ...
2024-12-27        13.1     B
2024-12-28        11.9     B
2024-12-29        12.9     B
2024-12-30        12.9     B
2024-12-31        12.1     B

[42249 rows x 2 columns]

In [9]:
df_full['05BB001']['FLOW']

1909-05-01     NaN
1909-05-02     NaN
1909-05-03     NaN
1909-05-04     NaN
1909-05-05     NaN
              ... 
2024-12-27    13.1
2024-12-28    11.9
2024-12-29    12.9
2024-12-30    12.9
2024-12-31    12.1
Freq: D, Name: FLOW, Length: 42249, dtype: float64

In [10]:
obj = {
    'type': 'QO',
    'location': coords,
    'timeseries': df_full['05BB001']['FLOW'],
    'units': 'm3/s',
    'freq': '1D',
}